## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/hack_data.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "hack_data_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `hack_data_csv`

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "hack_data_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
import pyspark

In [0]:
data=spark.read.csv("/FileStore/tables/hack_data.csv",inferSchema=True,header=True)

In [0]:
data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
only showing top 20 rows

In [0]:
from pyspark.ml.feature import VectorAssembler,StandardScaler

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
data.columns

Out[6]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [0]:
vector=VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'],outputCol='features')

In [0]:
final_data=vector.transform(data)

In [0]:
final_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed| features|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|[8.0,391.09,1.0,2...|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|[20.0,720.99,0.0,...|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|[31.0,356.32,1.0,...|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|[2.0,228.08,1.0,2...|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|[20.0,408.5,0.0,3...|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|[1.0,390.69,1.0,2...|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|[18.0,342.97,1.0,...|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|[22.0,101.61,1.0,...|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|[15.0,275.53,1.0,...|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|[12.0,424.83,1.0,...|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|[15.0,249.09,1.0,...|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|[32.0,242.48,0.0,...|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|[23.0,514.54,0.0,...|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|[9.0,284.77,0.0,3...|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|[27.0,779.25,1.0,...|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|[12.0,307.31,1.0,...|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|[21.0,355.94,1.0,...|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|[10.0,372.65,0.0,...|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|[20.0,347.23,1.0,...|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|[22.0,456.57,0.0,...|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
only showing top 20 rows

In [0]:
scalar=StandardScaler(inputCol='features',outputCol='scaledfeatures', withStd=True, withMean=False)

In [0]:
final_data=scalar.fit(final_data).transform(final_data)

In [0]:
final_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed| features| scaledfeatures|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|[20.0,720.99,0.0,...|[1.41962771166263...|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|[31.0,356.32,1.0,...|[2.20042295307707...|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|[2.0,228.08,1.0,2...|[0.14196277116626...|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|[20.0,408.5,0.0,3...|[1.41962771166263...|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|[1.0,390.69,1.0,2...|[0.07098138558313...|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|[18.0,342.97,1.0,...|[1.27766494049636...|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|[22.0,101.61,1.0,...|[1.56159048282889...|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|[15.0,275.53,1.0,...|[1.06472078374697...|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|[12.0,424.83,1.0,...|[0.85177662699757...|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|[15.0,249.09,1.0,...|[1.06472078374697...|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|[32.0,242.48,0.0,...|[2.27140433866020...|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|[23.0,514.54,0.0,...|[1.63257186841202...|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|[9.0,284.77,0.0,3...|[0.63883247024818...|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|[27.0,779.25,1.0,...|[1.91649741074455...|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|[12.0,307.31,1.0,...|[0.85177662699757...|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|[21.0,355.94,1.0,...|[1.49060909724576...|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|[10.0,372.65,0.0,...|[0.70981385583131...|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|[20.0,347.23,1.0,...|[1.41962771166263...|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|[22.0,456.57,0.0,...|[1.56159048282889...|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
modelk2=KMeans(featuresCol='scaledfeatures',k=2)
modelk3=KMeans(featuresCol='scaledfeatures',k=3)

In [0]:
modelk2_fit=modelk2.fit(final_data)

In [0]:
modelk3_fit=modelk3.fit(final_data)

In [0]:
modelk2_fit.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+

In [0]:
modelk3_fit.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+